In [8]:
from numpy.core.numeric import infty
!pip install numpy
!pip install pandas
!pip install plotly

!pip install scikit-learn # non-depreaceated sklearn

Task 1

In [9]:
import pandas as pd
# read the csv and make it a pandas.DataFrame
frame = pd.DataFrame(pd.read_csv("./online_shoppers_intention.csv"))
frame.describe(include='all') # 18 columns, including 2 categorical: Month & VisitorType, and two boolean: Weekend & Revenue

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
count,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330,12330.000000,12330.000000,12330.000000,12330.000000,12330,12330,12330
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,NaN,3,2,2
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,May,NaN,NaN,NaN,NaN,Returning_Visitor,False,False
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3364,NaN,NaN,NaN,NaN,10551,9462,10422
mean,2.315166,80.818611,0.503569,34.472398,31.731468,1194.746220,0.022191,0.043073,5.889258,0.061427,NaN,2.124006,2.357097,3.147364,4.069586,NaN,NaN,NaN
std,3.321784,176.779107,1.270156,140.749294,44.475503,1913.669288,0.048488,0.048597,18.568437,0.198917,NaN,0.911325,1.717277,2.401591,4.025169,NaN,NaN,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,1.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN
25%,0.000000,0.000000,0.000000,0.000000,7.000000,184.137500,0.000000,0.014286,0.000000,0.000000,NaN,2.000000,2.000000,1.000000,2.000000,NaN,NaN,NaN
50%,1.000000,7.500000,0.000000,0.000000,18.000000,598.936905,0.003112,0.025156,0.000000,0.000000,NaN,2.000000,2.000000,3.000000,2.000000,NaN,NaN,NaN
75%,4.000000,93.256250,0.000000,0.000000,38.000000,1464.157214,0.016813,0.050000,0.000000,0.000000,NaN,3.000000,2.000000,4.000000,4.000000,NaN,NaN,NaN


Verhaaltje van ~1min

In [10]:
#visualisation
# split the Browser 13 users from the rest
frame13 = frame.loc[frame.Browser == 13]
frameOther = frame.loc[frame.Browser != 13]

import plotly.graph_objects as go
# make a small dataframe with normalized data for plotting
def create_distribution(frameColumn):
    return pd.DataFrame({
        'Browser 13': frame13[frameColumn].value_counts(normalize=True),
        'Other browsers': frameOther[frameColumn].value_counts(normalize=True)
    }).reset_index().rename(columns={'index': col})
# plot all these columns
for col in ['VisitorType', 'Weekend', 'Revenue']:
    distribution = create_distribution(col)
    fig = go.Figure()
    fig.add_trace(go.Bar(x=distribution[col], y=distribution['Browser 13'], name='Browser 13'))
    fig.add_trace(go.Bar(x=distribution[col], y=distribution['Other browsers'], name='Other browsers'))
    fig.update_layout(barmode='group', title=f'{col} Distribution')
    fig.show()

Task 2

In [11]:
#Prepositioning op basis van algoritmes in task 3
# Preprocess using dropna AND change all string types to an integer
def Month_to_num(x):
    months = {
        "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4,
        "May": 5, "June": 6, "Jul": 7, "Aug": 8,
        "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12, "Returning_Visitor": 1, "Old_Visitor": 2, "New_Visitor": 3, "Other": 4, True: 1, False: 0}
    if x not in months:
        raise ValueError(f"Invalid month string: {x}")
    return months[x]


addlist = []
preprocessed = (frame.copy().dropna())
for i, j in enumerate(preprocessed['Month']):
    addlist.append(Month_to_num(j))
preprocessed['Month'] = addlist

addlist = []
for i, j in enumerate(preprocessed['VisitorType']):
    addlist.append(Month_to_num(j))
preprocessed['VisitorType'] = addlist

addlist = []
for i, j in enumerate(preprocessed['Weekend']):
    addlist.append(Month_to_num(j))
preprocessed['Weekend'] = addlist

addlist = []
for i, j in enumerate(preprocessed['Revenue']):
    addlist.append(Month_to_num(j))
preprocessed['Revenue'] = addlist

for i in preprocessed.columns:
    preprocessed[i] = pd.to_numeric(preprocessed[i], errors='raise')

print(preprocessed)




       Administrative  Administrative_Duration  Informational  \
0                   0                      0.0              0   
1                   0                      0.0              0   
2                   0                      0.0              0   
3                   0                      0.0              0   
4                   0                      0.0              0   
...               ...                      ...            ...   
12325               3                    145.0              0   
12326               0                      0.0              0   
12327               0                      0.0              0   
12328               4                     75.0              0   
12329               0                      0.0              0   

       Informational_Duration  ProductRelated  ProductRelated_Duration  \
0                         0.0               1                 0.000000   
1                         0.0               2                64.000000 

Task 3

In [12]:
# affinity propagation clustering - aan de hand van documentatie - NU HEEL SLOOM
from sklearn.cluster import AffinityPropagation, DBSCAN, Birch
Data = preprocessed.copy()
#Gebruik van sample maakt het proces sneller
Sample = Data.sample(n=1000)
clustering = AffinityPropagation(random_state=5).fit(Sample)
AffinityPropagation(random_state=24)

labels = clustering.labels_
cluster_centers = clustering.cluster_centers_
n_clusters = len(cluster_centers)

print("Number of clusters:", n_clusters)
print("Cluster centers:\n", cluster_centers)


# DBSCAN clustering
#TODO werkt maar de label -1 laat zien dat er een hoop data punten in ruis categorie vallen...
clustering2 = DBSCAN(eps=20).fit(Data)
labels2 = clustering2.labels_
print("Labels DBSCAN", labels2[:50])   

#Lijkt me dat beide problemen hier liggen aan de variabelen in de functies

# Birch clustering
#TODO nog een groot aantal clusters...
clustering3 = Birch(n_clusters=None).fit(Data)
labels3 = clustering3.labels_
print("Labels Birch", labels3[:50])   
aantal_clusters = len(set(labels3))
print("Aantal clusters Birch", aantal_clusters)


/Users/school/Documents/GitHub/DataAnalytics/.venvPytorc/lib/python3.8/site-packages/sklearn/cluster/_affinity_propagation.py:142: ConvergenceWarning:

Affinity propagation did not converge, this model may return degenerate cluster centers and labels.



Number of clusters: 67
Cluster centers:
 [[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.00000000e+00 8.79166667e+01 1.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.00000000e+01 1.25120000e+03 7.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [3.00000000e+00 4.10000000e+01 0.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]]
Labels DBSCAN [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 26  0  0  0 -1  0  0  0  0
  0  0  1  0  0 -1  0  0  0  0  0 20  0  0  0  0 -1  0  0  0  0  0  0  0
  0  0]
Labels Birch [  497  1743   344   154  5836  2063   461   155  1000  7110  4307  5519
  3884  2393  1817 10980   345  4473  1019 11566  2163   156  2437   724
   422  2296 10720  1429  1744 1

Task 4

In [ ]:
# manual Silhouette score, is zo volgensmij goed. Moet alleen zoeken hoe ik deze waardes kan pakken uit de labels
import numpy as np
def Silhouette_score(point, labelSELF, labels, Data):
    clusterSELF = Data[labels == labelSELF].to_numpy()
    clusters = Data[labels != labelSELF].to_numpy()
    afstand = 0

    for points in clusterSELF:
        if set(points) == set(point):
            continue
        afstand += np.linalg.norm(point-points)
    a = afstand/len(clusterSELF)

    lowest = infty
    clustr = None
    for clust in clusters:
        afstand_nearest = 0
        for points in clust:
            afstand_nearest += np.linalg.norm(point-points)
        dis = afstand_nearest/len(clust)
        if dis < lowest:
            lowest = dis
            clustr = clust
    b = lowest
    return (b-a)/(max(b,a))

def Silhouette_scores(x, labels):
    n = len(x)
    sil_values = []
    for i in range(n):
        sil_values.append(Silhouette_score(point=x.iloc[i].to_numpy(), labelSELF = labels[i], labels = labels, Data= x))
    print("Silhouette score:", sil_values)

from sklearn.metrics import davies_bouldin_score
print("Silhouette scores:")
print(f"Affinity propogation: {Silhouette_scores(Sample, labels)}")
print(f"DBSCAN: {Silhouette_scores(Data, labels2)}")
print(f"Birch Clustering: {Silhouette_scores(Data, labels3)}")
print()

# Davies Buildin Score
from sklearn.metrics import davies_bouldin_score
print("Davies Bouldin score:")
print(f"Affinity propogation: {davies_bouldin_score(Sample, labels)}")
print(f"DBSCAN: {davies_bouldin_score(Data, labels2)}")
print(f"Birch Clustering: {davies_bouldin_score(Data, labels3)}")
print()
# calinski-harabasz index
from sklearn.metrics import calinski_harabasz_score
print("Calinski Harabasz score:")
print(f"Affinity propogation: {calinski_harabasz_score(Sample, labels)}")
print(f"DBSCAN: {calinski_harabasz_score(Data, labels2)}")
print(f"Birch Clustering: {calinski_harabasz_score(Data, labels3)}")

Silhouette scores:


Task 5 ALL MANUAL


In [12]:
# Euclidian distance

# Manhattan distance

# Cosine similarity

# Distances effect on dbscan